In [2]:
import os #for navigation to the data folder 
import cv2 # image processing library
import numpy as np  #Linear Algebra Operations.
import pandas as pd   # Data manipulation
from sklearn.preprocessing import LabelEncoder#target labelling

image_folder = "D:\Agricultural_Dataset"
csv_file_path = "C:/Users/wam4e/OneDrive/Desktop/Features_Reduced.csv"
image_vectors = []#initializing lists 
labels = []#corresponding label
#image preprocessing: resizing, flattening and appending in the list
for category_index, category_name in enumerate(os.listdir(image_folder)):  #traverse through parent folder
    category_path = os.path.join(image_folder, category_name)   # for every sub-folder in parent folder
    
    for image_name in os.listdir(category_path):    # traverse through sub-folder
        image_path = os.path.join(category_path, image_name) # for each image in subfolder
        
        # Readinf and resizing all the images to the same shape mentioned.
        image = cv2.imread(image_path)  #first read image with cv2 library
        image = cv2.resize(image, (224, 224))  #then resize it
        
        # Vectorizing the image into a vector
        image_vector = image.flatten()  #then flatten it
        
        # Appending the vectorized image and corresponding target value/category to the lists
        image_vectors.append(image_vector)  #vectorize it(i.e storing one column under another column)
        labels.append(category_index)  #appending the catogerory to that image 

        
image_vectors = np.array(image_vectors)  # with numpy changing into arrays for numerical calucations
labels = np.array(labels)  # smae for labels 

label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)  # used to transform all numerical values into certain range by dividing all
                                                    # input with max among all inputs

# Creating a DataFrame with columns 'category' and 'x0', 'x1', 'x2', 'x3'....
column_names = ['category'] + [f'x{i}' for i in range(image_vectors.shape[1])]
data = np.column_stack((labels_encoded, image_vectors))
df = pd.DataFrame(data, columns=column_names)

# Saving the DataFrame(features and target) to the CSV file whose path is mentined above.
df.to_csv(csv_file_path, index=False)

In [4]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split # this function splits data for traning and testing parts 
from sklearn.preprocessing import StandardScaler  #it standardises the input and prevents some features from dominating others 
                                                 #due to their larger range of values.
from sklearn.metrics import accuracy_score  #this function is used to calculate accuracy

# Load the dataset
data = "C:/Users/wam4e/OneDrive/Desktop/Features_Reduced.csv"
df = pd.read_csv(data)  #loads into dataframe

# Preprocess the data
X = df.drop(['category'], axis=1)  #X drops category column and now contains only the feature columns
y = df['category'].values  #category values are stored in y

# Standardize the features
scaler = StandardScaler()  #Standardization ensures that all feature values have a mean of zero and a standard deviation of one
                        # which is often necessary for machine learning algorithms to work effectively.
X = scaler.fit_transform(X) #This step ensures that the features are on the same scale


# One-hot vector to predict the category of an input vector.
def one_hot_encode(labels, num_classes):
    num_samples = len(labels)
    encoded_labels = np.zeros((num_samples, num_classes))  # it creates numpy array
    for i in range(num_samples):
        encoded_labels[i, labels[i]] = 1  #only for that one class it sets to 1
    return encoded_labels

y = one_hot_encode(y, num_classes=8)

# Split the data into training and testing sets
# 85% images -> Training , 15% images -> Testing
Training_Data_Features, Testing_Data_Features, Target_Training_Data, Target_Testing_Data = train_test_split(X, y, 
                                                                                        test_size=0.15, random_state=42)


# Initialize weights and biases in all the layers
def initialize_parameters(input_size, hidden_size, output_size):
    W1 = np.random.randn(hidden_size, input_size) #initilising w1 with random number for first hidden layer
    b1 = np.zeros((hidden_size, 1))  # biase can be set to 0 or any small values
    W2 = np.random.randn(hidden_size, hidden_size) #initilising w2 with random number for second hidden layer
    b2 = np.zeros((hidden_size, 1))
    W3 = np.random.randn(output_size, hidden_size)  #initilising w3 with random number for output layer
    b3 = np.zeros((output_size, 1))
    return W1, b1, W2, b2, W3, b3



# Sigmoid activation function used for introducing "non-linearity" in "intermediate" hidden layers.
def sigmoid(z):
    return 1 / (1 + np.exp(-z))



# Forward propagation (feed-forward to obtain output).
def forward_propagation(X, W1, b1, W2, b2, W3, b3):
    Z1 = np.dot(W1, X) + b1
    A1 = sigmoid(Z1)  #sigmoid used for binary class classification
    Z2 = np.dot(W2, A1) + b2
    A2 = sigmoid(Z2)
    Z3 = np.dot(W3, A2) + b3
    
    
    # Soft-max function at the output layer.
    # 
    A3 = np.exp(Z3) / np.sum(np.exp(Z3), axis=0)  # soft-max hepls in multi-class classification and provides probability to 
                                                # which class it belongs
    return A1, A2, A3



# Compute the cross-entropy loss (function to describe loss/error in multi-class classification)
def compute_loss(A3, Y):
    m = Y.shape[1]
    loss = -1/m * np.sum(Y * np.log(A3))  #cross-entropy formula
    return loss



# Backpropagation with regard to the "cross-entropy loss function".
def backward_propagation(X, Y, A1, A2, A3, W2, W3):
    m = X.shape[1]
    # Compute gradients
    
    # For the last layer
    dZ3 = A3 - Y
    dW3 = (1/m) * np.dot(dZ3, A2.T)
    db3 = (1/m) * np.sum(dZ3, axis=1, keepdims=True)
    
    #For the second-last layer
    dA2 = np.dot(W3.T, dZ3)
    dZ2 = dA2 * A2 * (1 - A2)
    dW2 = (1/m) * np.dot(dZ2, A1.T)
    db2 = (1/m) * np.sum(dZ2, axis=1, keepdims=True)
    
    #for the the first layer
    dA1 = np.dot(W2.T, dZ2)
    dZ1 = dA1 * A1 * (1 - A1)
    dW1 = (1/m) * np.dot(dZ1, X.T)
    db1 = (1/m) * np.sum(dZ1, axis=1, keepdims=True)

    # derivatives correswponding to each layer.
    return dW1, db1, dW2, db2, dW3, db3



# Update parameters using gradient descent algorithm.
def update_parameters(W1, b1, W2, b2, W3, b3, dW1, db1, dW2, db2, dW3, db3, learning_rate):
    
    # Updating weight and bias term in first layer.
    W1 = W1 - learning_rate * dW1
    b1 = b1 - learning_rate * db1
    
    # Updating weight and bias in second layer.
    W2 = W2 - learning_rate * dW2
    b2 = b2 - learning_rate * db2
    
    # Updating weight and bias in third layer.
    W3 = W3 - learning_rate * dW3
    b3 = b3 - learning_rate * db3
    
    return W1, b1, W2, b2, W3, b3



# Train the neural network
def train_neural_network(X, Y, input_size, hidden_size, output_size, learning_rate, num_iterations):
    W1, b1, W2, b2, W3, b3 = initialize_parameters(input_size, hidden_size, output_size)
    losses = []

    for i in range(num_iterations):
        A1, A2, A3 = forward_propagation(X, W1, b1, W2, b2, W3, b3)
        loss = compute_loss(A3, Y)
        dW1, db1, dW2, db2, dW3, db3 = backward_propagation(X, Y, A1, A2, A3, W2, W3)
        W1, b1, W2, b2, W3, b3 = update_parameters(W1, b1, W2, b2, W3, b3, dW1, db1, dW2, db2, dW3, db3, learning_rate)
        losses.append(loss)

        if i % 10 == 0:
            print(f"Iteration {i}: Loss = {loss}")

    return W1, b1, W2, b2, W3, b3, losses



# Make predictions using the trained model.
def predict(X, W1, b1, W2, b2, W3, b3):
    _, _, A3 = forward_propagation(X, W1, b1, W2, b2, W3, b3)
    predictions = np.argmax(A3, axis=0)
    return predictions



# Testing the neural network and calculating accuracy.
def test_neural_network(Testing_Data_Features, Target_Testing_Data, W1, b1, W2, b2, W3, b3):
    predictions = predict(Testing_Data_Features, W1, b1, W2, b2, W3, b3)
    true_labels = np.argmax(Target_Testing_Data, axis=0)
    accuracy = accuracy_score(true_labels, predictions)
    return accuracy



# Define hyperparameters
input_size = Training_Data_Features.shape[1]
print("Size of Feature Vector = ", input_size)
print("Number of Hidden Layers = 3")
hidden_size = 256
print("Size of each Hidden Layer = ", hidden_size)
output_size = 8
print("Size of Output Layer = ", output_size)
learning_rate = 0.01
print("learning rate = ", learning_rate)
num_iterations = 100
print("Number of iterations = ", num_iterations)



# Train the neural network
W1, b1, W2, b2, W3, b3, losses = train_neural_network(Training_Data_Features.T, Target_Training_Data.T, input_size, 
                                                      hidden_size, output_size, learning_rate, num_iterations)



Size of Feature Vector =  150528
Number of Hidden Layers = 3
Size of each Hidden Layer =  256
Size of Output Layer =  8
learning rate =  0.01
Number of iterations =  100


C:\Users\wam4e\AppData\Local\Temp\ipykernel_20664\4010968531.py:50: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))


Iteration 0: Loss = 18.80524899074016
Iteration 10: Loss = 13.619800476294053
Iteration 20: Loss = 10.676067378471437
Iteration 30: Loss = 8.794833378307967
Iteration 40: Loss = 7.635417035342546
Iteration 50: Loss = 6.956629814848293
Iteration 60: Loss = 6.502724880637368
Iteration 70: Loss = 6.134145244644582
Iteration 80: Loss = 5.885606249739013
Iteration 90: Loss = 5.677454057898319


In [5]:
test_accuracy = test_neural_network(Testing_Data_Features.T, Target_Testing_Data.T, W1, b1, W2, b2, W3, b3)
print("Test Accuracy: {:.2f}%".format(test_accuracy * 100))

C:\Users\wam4e\AppData\Local\Temp\ipykernel_20664\4010968531.py:50: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))


Test Accuracy: 6.25%
